# Modélisation De Thématique 

Objectifs : 

Dans ce notebook l'objectif sera d'extraire des thématiques présentes dans les comptes rendus. Nous traiterons le corpus dans son entièreté. 

De fait, nous solliciterons : 

- Des modules de nettoyage de la librairie Spacy 

- La librairie de Gensim pour la modélisation en LDA 

- La librairie de pyLDAvis pour la visualisation des thématiques

Présentation du jeu de données : sortie.txt à partir duquel nous travaillons ici. 

sortie.txt est un fichier sous format txt contenant l'entièreté des comptes-rendus fournit. Il contient de fait l'entièreté des textes et sera ici transformer afin que chaque phrases devient un document à partir desquels nous réaliserons la LDA.

Dans un second temps nous couperons ce document en trois parties et réaliserons une LDA pour chacune des parties. 

PLAN : 

1. Modélisation de thématiques :
    1.1 Importation des données 
    1.2 Néttoyage 
2. Modèles LDA 
3. Visualisations 
4. Séparation en trois parties :
    4.1 Néttoyage 
    4.2 LDA et Visualisation :
5. Conclusion : 

    

# Importation et Installation :

### Installation pip : 

In [1]:
!pip install fr_core_web_sm

ERROR: Could not find a version that satisfies the requirement fr_core_web_sm (from versions: none)
ERROR: No matching distribution found for fr_core_web_sm
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install gensim==4.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20444 sha256=e946f507f8ded4cd210f9ef5d9c3b4648be30711c19e478fe6a7ddbaa5482dc5
  Stored in directory: /root/.cache/pip/wheels/99/66/48/d7ce0c6927f6abf167bbcdee537affc7b92c03632f78028411
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3529 sha256=e456c9d6ab543b9ee8e209d59afceaa4fd8a56eb00c6089d058b9d299f925902
  Stored in directory: /root/.cache/pip/wheels/d9/c7/71/db1d4646d963b34c530667501d3d6f34c0825eaffae2f0f2cb
Successfully built fst-pso miniful
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install pyLDAvis==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [5]:
!pip install wordcloud==1.8.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.6/458.6 KB 33.7 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


### Importation des librairies : 

In [129]:
# __IMPORTATION DES LIBRAIRES__
import os
import re
import numpy as np
import pandas as pd

# __NETTOYAGE :__ 

# Spacy pour lemmatisation 
import spacy

 # NLTK pour Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('french')

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

## spacy for lemmatization
import spacy
import fr_core_news_sm
nlp = fr_core_news_sm.load()
 
# __FONCTIONEMENT DES MODÈLES LDA :__
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint

 #__ VISUALISATION :__ 

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

#wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Importation du jeu de donnés sortie.txt

In [9]:
# Lecture des données, fichier texte simple : 
texte = ""
f = open("sortie.txt", "r", encoding='utf8')# r= read
for line in f.readlines():
    texte = texte + " " + line.rstrip("\n")

In [10]:
# Observation des caractéristiaques du tableau : 
print(type(texte))
print(len(texte))

<class 'str'>
8474179


# Préparation des données :

In [16]:
# Découpage du texte en paragraphe : 
print("Voici un extrait du texte à analyser : ","\n", texte[300:1700])
print(" ")
print("Notre texte initiale comporte", len(texte),"caractères.")

doc_set = re.split(("\."), texte)

print(" ")
print("Notre corpus de documents se comporte de environ", len(doc_set),"documents, ici des phrases.")

Voici un extrait du texte à analyser :  
  est consacrée à cette partie de YAppendix Vergiliana généralement publiée sous le nom de Catalecta ou de Catalepta. M. Galletier commence par établir que ces deux noms ne signifient rien, que ni l'un ni l'autre n'a jamais été le titre du recueil : Suétone, dans sa biographie de Virgile, a dû écrire quelque chose comme fecit Kaxà Xe-TCTÔv Priapea, Epigrammata, Diras..., et les deux mots grecs, transcrits en latin et plus ou moins défigurés, auront été pris pour le nom d'un recueil de vers. M. Galletier pense qu'il faut distinguer entre les Priapea (les trois premières pièces) et les Epigrammata (les 16 autres, ou pour mieux dire les15 autres, car le quatrain Callida imago est certainement interpolé). Les Priapées ont dû être publiées au temps de Domitien, les Épigrammes au temps de Néron.  Les Priapées, qui font de larges emprunts à la phraséologie virgilienne, ne peuvent être de Virgile ; elles contiennent même des réminiscences de Tibulle. El

In [17]:
# Création d'un dataframe pour le texte découpé en phrases : 
df=pd.DataFrame(doc_set)
df.rename( columns={0 :'texte'}, inplace=True )
df

,texte
0,CR n° 1 TITRE (P
1,Vergili Maronis)
2,Epigrammata et Priapea
3,Édition critique et explicative par E
4,Galletier TEXTE (P
...,...
77091,"— Ottokar Wreber, Die heilige Allianz, das Sy..."
77092,Plusieurs des articles sont illustrés
77093,H
77094,D


In [18]:
# Création d'une liste à partir des colonnes qui contiennent les textes à analyser :
var_texte = 'texte'
doc_set = df[var_texte].tolist() 

# Fonction pour Tokensiser la liste  : 
def sent_to_words(phrases):
    for mots in phrases:
        yield(simple_preprocess(str(mots), deacc=True))  # deacc=True removes punctuations

# Construction du corpus, liste des mots tokenisés :

data_words = list(sent_to_words(doc_set))

print ("Nos obtenos bien une liste avec tout nos documents : ",len(data_words))
print (type(data_words))

Nos obtenos bien une liste avec tout nos documents :  77096
<class 'list'>


In [19]:
#Apperçut de la liste
print(data_words[105])
print(data_words[106])
print(data_words[107])

['alexandre', 'kohut', 'fut', 'un', 'rabbin', 'qui', 'se', 'distingua', 'comme', 'predicateur', 'et', 'comme', 'orientaliste', 'et', 'qui', 'est', 'surtout', 'connu', 'par', 'un', 'dictionnaire', 'encyclopedique', 'du', 'talmud', 'aruch', 'completum', 'reedite', 'vienne', 'en']
['pour', 'honorer', 'sa', 'memoire', 'sa', 'famille', 'institue', 'plusieurs', 'fondations', 'en', 'vue', 'du', 'progres', 'des', 'etudes', 'juives']
['une', 'entre', 'elles', 'qui', 'date', 'de', 'celle', 'de', 'laquelle', 'depend', 'la', 'publication', 'dont', 'le', 'premier', 'fascicule', 'paru', 'paris', 'chez', 'geuthner', 'est', 'ici', 'annonce', 'deja', 'permis', 'edition', 'de', 'divers', 'textes', 'ou', 'recherches', 'sous', 'la', 'direction', 'du', 'departement', 'semitique', 'de', 'universite', 'de', 'yale']


## Nettoyage des données :

### Suppressions des mots outils : 

In [130]:
# Stopwords : 
print("stopwords importés :",len(stop_words))

#Raffinement des stopwords, complétée au fur et à mesure  des essai : 
stop_words.extend(["of","the","as","to","see","say","and","to", "comme","celle", "ici", "deja", "sous", "chez","tout","ce","ci","aussi", "dont", "elles","mem", "yal","cela","etc","celui","lui","ii","ete","ainsi","entre",'jusqu'])
print("stopwords complétés :",len(stop_words))



stopwords importés : 157
stopwords complétés : 188


In [29]:
# Functions pour nettoyage stopwords  : 
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [38]:
# Suppresion des mots outils :
data_remove_stopwords = remove_stopwords(data_words)

# Observation si elle marche bien : 
print("Les documents sans les stopwords :")
print(data_remove_stopwords[105])
print(data_remove_stopwords[106])
print(data_remove_stopwords[107])
print(data_remove_stopwords[206])

print("\n""Les documents avec les stopwords :")
print(data_words[105])
print(data_words[106])
print(data_words[107])
print(data_words[206])

Les documents sans les stopwords :
['alexandre', 'kohut', 'rabbin', 'distingua', 'predicateur', 'orientaliste', 'surtout', 'connu', 'dictionnaire', 'encyclopedique', 'talmud', 'aruch', 'completum', 'reedite', 'vienne']
['honorer', 'memoire', 'famille', 'institue', 'plusieurs', 'fondations', 'vue', 'progres', 'etudes', 'juives']
['date', 'laquelle', 'depend', 'publication', 'premier', 'fascicule', 'paru', 'paris', 'geuthner', 'annonce', 'permis', 'edition', 'divers', 'textes', 'recherches', 'direction', 'departement', 'semitique', 'universite', 'yale']
['correspondance', 'moscou', 'chilandar', 'poursuit', 'lettre', 'tsars', 'ivan', 'pierre', 'moines', 'chilandar']

Les documents avec les stopwords :
['alexandre', 'kohut', 'fut', 'un', 'rabbin', 'qui', 'se', 'distingua', 'comme', 'predicateur', 'et', 'comme', 'orientaliste', 'et', 'qui', 'est', 'surtout', 'connu', 'par', 'un', 'dictionnaire', 'encyclopedique', 'du', 'talmud', 'aruch', 'completum', 'reedite', 'vienne', 'en']
['pour', 'hon

### Lemmatization : 

In [39]:
# Lemmatization, réduction des mots à leur significations basique  :
def lemmatization(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [40]:
# Lemmatization :
data_lemmatized = lemmatization(data_remove_stopwords)

# Observer si elle s'applique bien 
print("\n""Les documents avant Lemmatisation :")
print(data_remove_stopwords[10])
print(data_remove_stopwords[130])

print("\n""Les documents après Lemmatisation :")
print(data_lemmatized[10])
print(data_lemmatized[130])


Les documents avant Lemmatisation :
['in', 'xv']
['aspetti', 'storici', 'tra', 'sei', 'settecento', 'texte', 'accademie', 'cultura']

Les documents après Lemmatisation :
['in', 'xv']
['aspetti', 'storici', 'tra', 'sei', 'settecento', 'text', 'accademie', 'culturer']


In [41]:
# Observer si elle s'applique bien
print("\n""Les documents avant Lemmatisation :")
print(data_remove_stopwords[105])
print(data_remove_stopwords[106])

print("\n""Les documents après Lemmatisation :")
print(data_lemmatized[105])
print(data_lemmatized[106])


Les documents avant Lemmatisation :
['alexandre', 'kohut', 'rabbin', 'distingua', 'predicateur', 'orientaliste', 'surtout', 'connu', 'dictionnaire', 'encyclopedique', 'talmud', 'aruch', 'completum', 'reedite', 'vienne']
['honorer', 'memoire', 'famille', 'institue', 'plusieurs', 'fondations', 'vue', 'progres', 'etudes', 'juives']

Les documents après Lemmatisation :
['alexandre', 'kohut', 'rabbin', 'distingua', 'predicateur', 'orientaliste', 'surtout', 'connaître', 'dictionnair', 'encyclopediqu', 'talmud', 'aruch', 'completum', 'reedite', 'venir']
['honorer', 'memoir', 'famill', 'institu', 'plusieurs', 'fondation', 'voir', 'progre', 'etuder', 'juif']


La légalisation transforme les mots à leur racine sémantique comme observé dans l'exemples si dessus : 

connu => connaître

etudes => etuder

vue => voir

Nous observons que la lemmatisation marche bien et que les mots sont réduit à leur signification sémantique.

# LDA 

### Préparation des données :

In [42]:
# Créattion du Dictionary :
dictio = corpora.Dictionary(data_lemmatized)
dictio = dictio.filter_extremes(no_below=10)

 #Il est recommandé d'utiliser une valeur de 10 pour l'argument no_below dans la commande dico.filter_extremes(). 
 #Cela garantit que tous les mots apparaissant moins de 10 fois seront retirés du dictionnaire donné pour être traités par LDA.

# Création du Corpus : 
texts = data_lemmatized

# Création de la matrise terme / document
corpus = [dictio.doc2bow(text) for text in texts]

#  Affichage :
print(len(dictio),type(dictio))
print(len(texts), type(texts), texts[106])

# Appercut du dictionaire :
[[(dictio[id], freq) for id, freq in cp] for cp in corpus[106:108]]

8342 <class 'gensim.corpora.dictionary.Dictionary'>
77096 <class 'list'> ['honorer', 'memoir', 'famill', 'institu', 'plusieurs', 'fondation', 'voir', 'progre', 'etuder', 'juif']


[[('plusieurs', 1),
  ('voir', 1),
  ('fondation', 1),
  ('etuder', 1),
  ('famill', 1),
  ('honorer', 1),
  ('institu', 1),
  ('juif', 1),
  ('memoir', 1),
  ('progre', 1)],
 [('edition', 1),
  ('texte', 1),
  ('pari', 1),
  ('premier', 1),
  ('publication', 1),
  ('geuthner', 1),
  ('annonce', 1),
  ('date', 1),
  ('departemer', 1),
  ('depend', 1),
  ('direction', 1),
  ('divers', 1),
  ('fascicul', 1),
  ('lequel', 1),
  ('paraître', 1),
  ('permettre', 1),
  ('recherche', 1),
  ('semitique', 1),
  ('universit', 1),
  ('yal', 1)]]

Le dictionaire nous semble contenir des mots "propres". Nous observons beaucoup moins de mots outils grâce à l'extension de la liste des stopwords plus haut. De même, les mots son Lemmatizés. De fait, ces données nous sembles exploitable pour une LDA. 

In [43]:
#Liste qui servira pour les WordClouds : 
lad_models =[]

### Lda_model_1, 5 thématiques : 

In [44]:
# Build LDA model5 thèmes
lda_model_1 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictio,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the topics
pprint(lda_model_1.print_topics())
#doc_lda = lda_model_1[corpus]

#Pour modele 1
from gensim.topic_coherence import probability_estimation

# A titre indicatif nous mettons le taux de perplexité, même si celui là ne sera pas retenus. 
print('\nPerplexity: ', lda_model_1.log_perplexity(corpus))

lad_models.append(lda_model_1)

[(0,
  '0.019*"celui" + 0.019*"lequel" + 0.014*"vase" + 0.012*"collection" + '
  '0.011*"lettre" + 0.010*"ordre" + 0.010*"contre" + 0.010*"ni" + '
  '0.010*"savant" + 0.010*"introduction"'),
 (1,
  '0.019*"saint" + 0.012*"figure" + 0.009*"vers" + 0.009*"roi" + '
  '0.007*"francai" + 0.007*"eglise" + 0.007*"raison" + 0.007*"long" + '
  '0.007*"hui" + 0.007*"aujourd"'),
 (2,
  '0.018*"plus" + 0.012*"ce" + 0.011*"faire" + 0.011*"pouvoir" + 0.008*"tout" '
  '+ 0.008*"siecle" + 0.008*"texte" + 0.008*"grand" + 0.008*"autre" + '
  '0.008*"auteur"'),
 (3,
  '0.016*"moins" + 0.016*"romain" + 0.012*"avoir" + 0.012*"leur" + 0.009*"vie" '
  '+ 0.008*"etat" + 0.007*"mot" + 0.007*"temp" + 0.007*"vouloir" + '
  '0.007*"politique"'),
 (4,
  '0.023*"der" + 0.015*"egypte" + 0.013*"die" + 0.012*"ensuite" + 0.012*"von" '
  '+ 0.012*"devenir" + 0.012*"und" + 0.012*"archeologie" + 0.012*"pendant" + '
  '0.010*"facon"')]

Perplexity:  -8.005199432527125


Test avec 5 thématiques premières sorties  : 

- num_topics=5

- random_state=100

- passes=100

Thématique 1 ( politique / historien / droit / contre) : Cette première thématique semble surtout ce qualifié par l'histoire et la politique ce qui nous semble assez logique et intuitifs puisque un grand nombre de comptes-rendus, du journal des savants s'inscrive dans des analyses historiques. 

[(0,
  '0.021*"entre" + 0.015*"mettre" + 0.014*"avoir" + 0.013*"leur" + 0.010*"der" '
  '+ 0.009*"prendre" + 0.008*"historien" + 0.008*"contre" + 0.008*"droit" + '
  '0.008*"politique"'),

Thématique 2 ( periode / saint / roi / religieux) : Cette deuxième les thématique semble faire référence à des sujet religieux ou liées à la royauté.  De même, nous identifiant une dimension temporelle avec un vocabulaire liées aux chronologies historiques comme "siècle" , "années" ou "périodes". 
 (1,
  '0.040*"siecle" + 0.023*"apre" + 0.020*"saint" + 0.012*"fin" + 0.010*"vie" + '
  '0.009*"religieux" + 0.009*"vers" + 0.009*"roi" + 0.009*"annee" + '
  '0.009*"periode"'),

Thématique 3 ( periode / saint / roi / religieux) : La troisième thématique ne nous semble pas véritablement parlante. En effet, elle contient des mot outils comme tout, qui ne sont pas passer aux nettoyage des textes. De même, nous pouvons pas véritablement dire si "pouvoir" fait ici référence aux verbe "pouvoir" ou alors du nom "le pouvoir". 


 (2,  '0.017*"plus" + 0.014*"tout" + 0.011*"ce" + 0.011*"faire" + 0.011*"pouvoir" '
  '+ 0.010*"celui" + 0.009*"dont" + 0.008*"texte" + 0.008*"grand" + '
  '0.008*"autre"'), 

Thématique 4 ( planche / style /groupe / vase) : Nous pouvons ici supposer que cette thématique fasse référence aux styles artistique. En se caractérisant par des mots comme groupe, temps et styles nous pouvons émettre l'hypothèse que la LDA identifie ici un penchant du Journal des Savant à s'intéresser aux expressions culturelle sous la forme d'expression esthétique. Cette hypothèse est fort plausible puisque l'analyse en archéologies, ou en histoire de l'art est longuement rester dans des approches formaliste. De même, nous remarquons ici que des erreurs persiste malgré les nettoyages. En effet, nous avons le s qui semble manquer à temps et même nous parait être devenus "mes". C'est là une de notre problématique centrale, le texte oceriser sembles résister à certains nettoyages automatique. Cela peut être du à des problématique d'encodage que nous avons pue préciser dans la partie sur l'analyse textuelles. 

(3, '0.020*"mem" + 0.015*"pi" + 0.014*"vase" + 0.012*"planche" + 0.011*"temp" + '
  '0.011*"etaient" + 0.008*"influence" + 0.008*"style" + 0.007*"groupe" + '
  '0.007*"notamment"'),

Thématique 5 (savant/ introduction/ pays/ fascicul / ecole/ societe/ source /maison ) : La cinquième thématique nous sembles refléter un résultat satisfaisant. Avec un vocabulaire comme source, école et savant cette thématique sembles souligner un point centrales des comptes-rendus, à savoir, la sollicitations de sources et de courant de pensées. En effet, cette thématique relève un point technique de rédaction intrinsèque aux comptes-rendu. 
 (4, '0.012*"an" + 0.011*"savant" + 0.010*"introduction" + 0.010*"pays" + '
  '0.010*"source" + 0.009*"louis" + 0.007*"maison" + 0.007*"fascicul" + '
  '0.006*"ecole" + 0.006*"societe"')]

On observe que la Perplexity est de  -7.423903616951168. Ce score à été utiliser pour évaluer des modèles cependant nous ne tiendrons pas comptes de celui-ci puisque il a été montrer que cet indicateur ne permet pas de véritablement évaluer les modèles. De fait, nous le signalons ici juste que la mesure existe. 

Si la premières sorties donnait des résultat assez parlant, la deuxième sorties donnent des résultat qui ne sont que difficilement interprétables. 

Thème 1 : collection, savant, lettre, introduction

[(0,
  '0.019*"celui" + 0.019*"lequel" + 0.014*"vase" + 0.012*"collection" + '
  '0.011*"lettre" + 0.010*"ordre" + 0.010*"contre" + 0.010*"ni" + '
  '0.010*"savant" + 0.010*"introduction"'),

Thème 2 : eglise, raison, roi, francai
 (1,
  '0.019*"saint" + 0.012*"figure" + 0.009*"vers" + 0.009*"roi" + '
  '0.007*"francai" + 0.007*"eglise" + 0.007*"raison" + 0.007*"long" + '
  '0.007*"hui" + 0.007*"aujourd"'),

Thème 3 : siecle, auteur, texte
 (2,
  '0.018*"plus" + 0.012*"ce" + 0.011*"faire" + 0.011*"pouvoir" + 0.008*"tout" '
  '+ 0.008*"siecle" + 0.008*"texte" + 0.008*"grand" + 0.008*"autre" + '
  '0.008*"auteur"'),

Thème 4 : romain, politique, etat
 (3,
  '0.016*"moins" + 0.016*"romain" + 0.012*"avoir" + 0.012*"leur" + 0.009*"vie" '
  '+ 0.008*"etat" + 0.007*"mot" + 0.007*"temp" + 0.007*"vouloir" + '
  '0.007*"politique"'),

Thème 5 : egypte, archeologie
 (4,
  '0.023*"der" + 0.015*"egypte" + 0.013*"die" + 0.012*"ensuite" + 0.012*"von" '
  '+ 0.012*"devenir" + 0.012*"und" + 0.012*"archeologie" + 0.012*"pendant" + '
  '0.010*"facon"')]

On peut ici dire que on a d'un coter des thèmes liées aux sujet d'études ( romain, Egypte) et de l'autre des thèmes liées au vocabulaire technique des comptes rendus comme ( auteur, texte, savant, lettre). 

### lda_model_2, 20 thématiques : 

In [45]:
# Train the LDA model
lda_model_2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictio,
                                           num_topics=20, 
                                           random_state=50,
                                           update_every=10,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

lad_models.append(lda_model_2)

# Print the Keyword in the topics

pprint(lda_model_2.print_topics())

[(0,
  '0.069*"ce" + 0.066*"faire" + 0.046*"grand" + 0.045*"auteur" + '
  '0.032*"premier" + 0.028*"eter" + 0.028*"apre" + 0.025*"encore" + '
  '0.025*"nouveau" + 0.025*"volume"'),
 (1,
  '0.059*"pi" + 0.048*"inscription" + 0.044*"lettre" + 0.042*"contre" + '
  '0.038*"source" + 0.037*"louis" + 0.033*"debut" + 0.031*"paul" + '
  '0.029*"esprit" + 0.028*"illustration"'),
 (2,
  '0.067*"tre" + 0.040*"mettre" + 0.032*"suivre" + 0.030*"rendre" + '
  '0.029*"question" + 0.027*"fort" + 0.027*"edition" + 0.024*"critique" + '
  '0.021*"ordre" + 0.020*"jour"'),
 (3,
  '0.074*"meme" + 0.070*"bien" + 0.056*"si" + 0.047*"quelque" + 0.045*"dire" + '
  '0.042*"peu" + 0.036*"certain" + 0.025*"falloir" + 0.023*"point" + '
  '0.022*"assez"'),
 (4,
  '0.154*"plus" + 0.067*"autre" + 0.059*"deux" + 0.044*"sans" + 0.033*"ancien" '
  '+ 0.031*"grec" + 0.027*"nom" + 0.024*"monument" + 0.022*"figure" + '
  '0.020*"important"'),
 (5,
  '0.072*"part" + 0.049*"ailleurs" + 0.045*"trouve" + 0.044*"epoque" + '
  '0

Interprétation : 

Avec 20 thématiques nous pouvons dégager les sujets suivants : 

Les thématiques, si dessous se caractérisent par un vocabulaire faisant références à des artefacts étudiés. Nous avons de fait la mention d'art, de sculptures, monument. Des mots comme figure ou fouilles peuvent présenter un certains degrés d'ambiguïtés. C'est une des caractéristique qui rend l'analyse du langage naturel délicates, puisque un mot peut toujours avoir plusieurs significations en fonction du contexte dans lequel il est utilisé. 

(4,
  '0.154*"plus" + 0.067*"autre" + 0.059*"deux" + 0.044*"sans" + 0.033*"ancien" '
  '+ 0.031*"grec" + 0.027*"nom" + 0.024*"monument" + 0.022*"figure" + '
  '0.020*"important"'), 

(15,
  '0.100*"avoir" + 0.058*"vi" + 0.038*"sculpture" + 0.035*"primitif" + '
  '0.030*"cas" + 0.029*"universite" + 0.029*"riche" + 0.026*"archive" + '
  '0.025*"comte" + 0.024*"conserve"'),

(16,
  '0.121*"histoire" + 0.083*"art" + 0.068*"etude" + 0.061*"chapitre" + '
  '0.049*"travail" + 0.042*"depuis" + 0.033*"premiere" + 0.030*"consacre" + '
  '0.029*"an" + 0.021*"fouille"'),

Les thématiques si dessous sembles se baser sur le vocabulaire technique relatifs aux comptes rendus. Avec des mots comme titre, text, vol (pour volume), analyse, ouvrages, sujet documents et livre, ces thèmes s'inscrive dans la tendance générale de l'analyse LDA qui dégage peut importe le nombre de thématiques sélectionné des thèmes caractérisant le comtes rendus sous sa forme méthodologique. 

(6,
  '0.187*"titre" + 0.136*"cr" + 0.095*"vol" + 0.072*"text" + 0.047*"historien" '
  '+ 0.032*"von" + 0.021*"egalement" + 0.018*"connaitre" + 0.018*"di" + '
  '0.017*"voici"'),

(8,
  '0.096*"ouvrage" + 0.053*"souvent" + 0.043*"droit" + 0.042*"ni" + '
  '0.041*"toujours" + 0.037*"reste" + 0.035*"presque" + 0.030*"analyse" + '
  '0.025*"technique" + 0.025*"type"'),

 (13,
  '0.056*"livre" + 0.032*"donne" + 0.031*"surtout" + 0.027*"sujet" + '
  '0.026*"document" + 0.026*"divers" + 0.025*"fin" + 0.024*"avant" + '
  '0.023*"enfin" + 0.021*"moyen"'),

### lda_model_3, 50 thématiques : 

In [46]:
# Train the LDA model
lda_model_3 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictio,
                                           num_topics=50, 
                                           random_state=50,
                                           update_every=10,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

lad_models.append(lda_model_3)

# Print the Keyword in the topics

pprint(lda_model_3.print_topics())

[(0,
  '0.120*"pays" + 0.107*"conclusion" + 0.087*"traire" + 0.047*"origin" + '
  '0.033*"souvenir" + 0.031*"vii" + 0.030*"ix" + 0.025*"demande" + '
  '0.025*"della" + 0.023*"abbe"'),
 (31,
  '0.135*"annee" + 0.104*"semble" + 0.065*"xvi" + 0.057*"deuxieme" + '
  '0.049*"merite" + 0.041*"bibliotheque" + 0.040*"indiquer" + 0.036*"richesse" '
  '+ 0.033*"auquel" + 0.029*"simplement"'),
 (46,
  '0.176*"figure" + 0.063*"lire" + 0.060*"sen" + 0.057*"memoir" + '
  '0.054*"interesser" + 0.051*"condition" + 0.043*"rare" + 0.043*"capital" + '
  '0.035*"presenter" + 0.035*"famill"'),
 (1,
  '0.203*"leur" + 0.117*"plusieurs" + 0.056*"terre" + 0.052*"comte" + '
  '0.049*"passage" + 0.047*"roman" + 0.041*"academie" + 0.030*"contemporain" + '
  '0.026*"forment" + 0.025*"yeux"'),
 (35,
  '0.227*"donne" + 0.093*"comprendre" + 0.073*"iii" + 0.050*"outre" + '
  '0.048*"philippe" + 0.044*"relation" + 0.041*"indication" + 0.023*"subir" + '
  '0.021*"transformation" + 0.020*"construire"'),
 (40,
  '0.098*"l

 Nous interpréterons ce modèle grâce à la visualisation qui permet de mieux représenter les thèmes que l'LDA à dégagé. 

### Lda_model_4, 7 thématiques : 

In [47]:
# Train the LDA model
lda_model_4 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictio,
                                           num_topics=7, 
                                           random_state=50,
                                           update_every=10,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

lad_models.append(lda_model_4)

# Print the Keyword in the topics

pprint(lda_model_4.print_topics())

[(0,
  '0.045*"premier" + 0.032*"chapitre" + 0.020*"second" + 0.019*"pi" + '
  '0.013*"rien" + 0.013*"pays" + 0.012*"hui" + 0.012*"aujourd" + 0.011*"bas" + '
  '0.011*"culte"'),
 (1,
  '0.041*"texte" + 0.021*"nouveau" + 0.020*"saint" + 0.016*"nom" + 0.015*"non" '
  '+ 0.011*"fort" + 0.010*"fait" + 0.010*"souvent" + 0.010*"seulement" + '
  '0.009*"inscription"'),
 (2,
  '0.034*"in" + 0.022*"volume" + 0.020*"pari" + 0.018*"vol" + 0.018*"page" + '
  '0.012*"venir" + 0.011*"general" + 0.011*"edition" + 0.010*"collection" + '
  '0.010*"consacre"'),
 (3,
  '0.028*"pouvoir" + 0.027*"faire" + 0.017*"autre" + 0.016*"etre" + 0.015*"si" '
  '+ 0.013*"art" + 0.013*"sans" + 0.012*"eter" + 0.012*"dire" + 0.011*"apre"'),
 (4,
  '0.036*"plus" + 0.022*"ce" + 0.017*"tout" + 0.017*"siecle" + 0.016*"grand" + '
  '0.015*"histoire" + 0.015*"meme" + 0.014*"bien" + 0.013*"deux" + '
  '0.011*"livre"'),
 (5,
  '0.018*"romain" + 0.014*"tel" + 0.012*"figure" + 0.010*"der" + '
  '0.010*"dernier" + 0.009*"vers" + 0

Nous interpréterons ce modèle grâce à la visualisation qui permet de mieux représenter les thèmes que l'LDA à dégagé. 

# Visualisation 

PyLDAvis est une bibliothèque qui vous aide à interpréter les modèles thématiques (LDA).On réalisera cette visualisation sur nos trois modèles.

### Visualisation de lda_model_1

In [48]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_1, corpus, dictio)

warnings.filterwarnings("ignore",category=DeprecationWarning)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.408131 -0.047093       1        1  59.395703
3      0.190124 -0.347705       2        1  13.982959
1      0.137618  0.296108       3        1  11.972189
0      0.069809  0.085994       4        1   9.675450
4      0.010580  0.012694       5        1   4.973699, topic_info=        Term         Freq        Total Category  logprob  loglift
38      plus  6683.000000  6683.000000  Default  30.0000  30.0000
949    saint  1460.000000  1460.000000  Default  29.0000  29.0000
36     moins  1412.000000  1412.000000  Default  28.0000  28.0000
168    celui  1182.000000  1182.000000  Default  27.0000  27.0000
664   romain  1372.000000  1372.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
6487  moreau   141.570858   142.478669   Topic5  -5.4024   2.9946
2109  malgre   124.566965   125.368444   Topic5  -5.5304   2.9946
1517    regn   117.820904   118.623588   Topic5  -5.5860   2.9942
258    civil   115.851004   116.651723   Topic5  -5.6029   2.9941
398    blanc   116.239684   117.053245   Topic5  -5.5995   2.9940

[182 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2093      5  0.993410  administration
318       2  0.997984            agir
1542      4  0.997893         ajouter
166       5  0.995487        allemand
707       2  0.997445           aller
...     ...       ...             ...
1924      5  0.998660             von
480       2  0.999341         vouloir
105       3  0.995964             xiv
601       5  0.997727             xvi
2475      5  0.993832             zur

[152 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1, 5])

Thème 1 : lié à l'études et la méthodologies des comptes rendus : "histoire, auteurs, livre, texte, sciecel, titre"

Thème 2 : lié à l'empire romain : "romain, état, politique, Rome, influence, études orientale" 

Thème 3: lié à la France : "royauté, styles, église, française" 

Thèmes 4 : lié à l'études et la conversation scientifique : "école indexe citer, analyse, écrit, savant" 

### Visualisation de lda_model_2

In [79]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_2, corpus, dictio)

warnings.filterwarnings("ignore",category=DeprecationWarning)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.304057 -0.348497       1        1  14.533915
4      0.189391  0.288646       2        1   9.899309
3      0.151008  0.131042       3        1   8.636872
13     0.127823  0.091014       4        1   8.357483
2      0.059320  0.025141       5        1   6.748890
10     0.064407  0.028454       6        1   6.401823
16     0.020745  0.006994       7        1   5.269074
17    -0.035897 -0.010285       8        1   3.981845
7     -0.052001 -0.014161       9        1   3.753601
9     -0.059000 -0.015082      10        1   3.359392
14    -0.070836 -0.017857      11        1   3.354327
1     -0.065962 -0.016745      12        1   3.302004
8     -0.070258 -0.017572      13        1   3.256341
18    -0.065833 -0.016444      14        1   3.218699
5     -0.070897 -0.017511      15        1   3.160125
11    -0.065955 -0.016339      16        1   3.071942
6     -0.067434 -0.016616      17        1   2.823554
15    -0.097437 -0.021587      18        1   2.451777
19    -0.088774 -0.020083      19        1   2.328239
12    -0.106469 -0.022512      20        1   2.090790, topic_info=          Term         Freq        Total Category  logprob  loglift
38        plus  9658.000000  9658.000000  Default  30.0000  30.0000
63     pouvoir  5851.000000  5851.000000  Default  29.0000  29.0000
5        texte  4457.000000  4457.000000  Default  28.0000  28.0000
100     siecle  4453.000000  4453.000000  Default  27.0000  27.0000
164       tout  4548.000000  4548.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2126     armee   126.509770   127.848926  Topic20  -4.6483   3.8571
459   distingu   118.157444   119.496600  Topic20  -4.7166   3.8564
2128     jusqu   117.182256   118.521411  Topic20  -4.7248   3.8563
1280   imprime   114.203317   115.542473  Topic20  -4.7506   3.8560
1285    annees   112.507963   113.847118  Topic20  -4.7656   3.8558

[630 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
434      13  0.995130   abondant
1244     10  0.998546      abord
866      16  0.991556    academi
730      12  0.995553   academie
820      13  0.993835  academiqu
...     ...       ...        ...
122      12  0.996002        xii
105      15  0.997829        xiv
8        14  0.996204         xv
601      14  0.997108        xvi
243       1  0.999359      œuvre

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 14, 3, 11, 17, 18, 8, 10, 15, 2, 9, 19, 6, 12, 7, 16, 20, 13])

Les 20 thématiques permettent d'identifiées : 

Des thématiques liées à l'études d'ouvrages et des sujet d'art et d'archéologies avec des mots comme (dans la thématique 7 par exemple) : "archéologie, information, chapitre, histoire, art, études, sociétés et fouilles ". Nous notons que dans cette thématiques des mots comme "lumières" peuvent être ambiguë, puisque il peut faire référence à une périodes historique, avoir une valeur allégorique comme descriptive. 

La thématique 1 est à l'écart ici, elle contient des mots comme "publication, auteur, oeuvre, nouveau volume "semble être liée à la thématique de parution d'ouvrage. Cela s'inscrirait totalement dans l'identité du compte rendus comme document analysant généralement les nouvelles parutions. 

Nous avons aussi des thèmes liées aux monde antique avec un vocabulaire faisant références aux "édifices" et à leurs "inscription",  au "monument", "ancien", "grec".  

Les résultat sembles présenter des modèles plus prometteurs quand à la compréhension de l'ensemble du corpus.  

### Visualisation de lda_model_3

In [49]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_3, corpus, dictio)

warnings.filterwarnings("ignore",category=DeprecationWarning)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
24     0.295661 -0.355475       1        1  7.273384
20     0.177257  0.291155       2        1  4.226094
45     0.110815  0.077572       3        1  3.226461
28     0.094269  0.057415       4        1  3.030827
22     0.085501  0.048931       5        1  2.850654
15     0.092885  0.056460       6        1  2.847452
39     0.085372  0.048939       7        1  2.812756
41     0.059357  0.027988       8        1  2.599309
5      0.047808  0.020954       9        1  2.490979
6      0.058281  0.027605      10        1  2.433922
26     0.039007  0.016139      11        1  2.320334
42     0.034047  0.014091      12        1  2.252506
47     0.026831  0.009910      13        1  2.218831
17     0.028366  0.011065      14        1  2.125916
33     0.018304  0.006729      15        1  2.092872
10     0.008860  0.002513      16        1  2.061250
14     0.003734  0.000313      17        1  2.019344
34     0.007696  0.001894      18        1  2.014418
13    -0.000218 -0.001222      19        1  2.006167
18    -0.004623 -0.001460      20        1  1.839300
16    -0.005932 -0.002076      21        1  1.821257
19    -0.005829 -0.002413      22        1  1.819390
44    -0.009986 -0.003803      23        1  1.771143
3     -0.021180 -0.007520      24        1  1.765753
27    -0.011467 -0.003945      25        1  1.753856
8     -0.013179 -0.004718      26        1  1.751109
21    -0.018753 -0.006756      27        1  1.750410
30    -0.014383 -0.004585      28        1  1.719745
4     -0.020150 -0.007025      29        1  1.715970
2     -0.023078 -0.007750      30        1  1.709977
12    -0.018581 -0.006161      31        1  1.694015
23    -0.030087 -0.010154      32        1  1.686523
11    -0.035182 -0.010993      33        1  1.608876
32    -0.044379 -0.013702      34        1  1.536696
49    -0.050994 -0.015172      35        1  1.491086
37    -0.044631 -0.013098      36        1  1.467645
29    -0.043555 -0.012827      37        1  1.462384
9     -0.042299 -0.012470      38        1  1.441614
38    -0.053869 -0.015406      39        1  1.415048
7     -0.045291 -0.013372      40        1  1.393092
36    -0.057638 -0.016091      41        1  1.364659
43    -0.064439 -0.017794      42        1  1.360653
25    -0.058530 -0.016305      43        1  1.352659
48    -0.065076 -0.017512      44        1  1.285357
40    -0.071780 -0.018903      45        1  1.262607
35    -0.076688 -0.019663      46        1  1.219894
1     -0.074866 -0.019444      47        1  1.212547
46    -0.075136 -0.019400      48        1  1.185254
31    -0.081939 -0.020608      49        1  1.144250
0     -0.090313 -0.021849      50        1  1.093755, topic_info=             Term         Freq        Total Category  logprob  loglift
38           plus  9799.000000  9799.000000  Default  30.0000  30.0000
60          faire  6043.000000  6043.000000  Default  29.0000  29.0000
63        pouvoir  5915.000000  5915.000000  Default  28.0000  28.0000
5           texte  4571.000000  4571.000000  Default  27.0000  27.0000
100        siecle  4377.000000  4377.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
499       thebain    78.918387    80.329047  Topic50  -4.4722   4.4978
1009          sir    73.968634    75.379294  Topic50  -4.5370   4.4967
1579        roger    72.987078    74.397738  Topic50  -4.5504   4.4964
1560  commencemer    70.074431    71.485092  Topic50  -4.5911   4.4956
1566       croise    69.445505    70.856165  Topic50  -4.6001   4.4954

[1549 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
464      45  0.972582   abandon
1551     31  0.984621     abbay
1552     50  0.991106      abbe
434      43  0.994095  abondant
1244     10  0.998804     abord
...     ...       ...       ...
925      30  0.971357      xvii
814      42  0.984754  

Avec un nombre élevé de thématiques, ici 50, nous remarquons que nous avons un grand nombres de thématiques qui sont similaires, mais qui se caractérises par quelques mots différents. Cela peut s'expliquer du fait que les comptes rendus ont un vocabulaire similaire due à leur méthodologies de rédactions et à cause du fait qu'ils s'appuient sur des sources similaires (documents, textes, vol, chapitre, oeuvres, etc). C'est ce vocabulaire qui engendre la superpositions des sphère sur la visualisation. Les mots différents comme "gaule, charles, mosaïque, arbre, monastere" dans le thème 33 vont alors le diferenciers et le caractériser plus spécialement. 


Avec des thèmes suivant : 

Thème 7 : "grec, latin, inscription, renseignement, index, chronologique, table, concordance, calendrier, ordre, librairie"

Thème 18 : "probleme, interprétation, hypothèse, bronze, terrain, decouvertes, archeologiquer, emplacement"

Thème 43 : "trouve, article, presente, artistique, parfaitement, systeme, apollon, reel, substantiel, possession, admirablemen, copier, possession, unique, muse"

Thème 33 : "imperial, impression, honneur, enseignement, gaule charles, mosaique, imprime arbre, catalogue, monastere, droite

Thème 16 : " connaître, savant, Rome, journal, scientifique, pape, christ, christianisme, porter, moral, exactitude, traduction, méthodes"

Nous avons l'impression que en réalisant une LDA pour un grand nombre de thématiques, cela permet d'identifier plus correctement de véritables thématiques. De fait, nous n'avons plus la grande scission entre d'un cotées les thèmes évoquant un vocabulaire technique et de l'autre quelques thématiques évoquant une périodes historique ou une disciplines. 


### Visualisation de lda_model_4

In [50]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_4, corpus, dictio)

warnings.filterwarnings("ignore",category=DeprecationWarning)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.299742 -0.004895       1        1  29.694819
3     -0.281777  0.005921       2        1  22.976912
5      0.145823 -0.231048       3        1  12.061329
1      0.139569 -0.110611       4        1  11.821779
2      0.135566  0.397757       5        1  11.261413
0      0.091786 -0.028781       6        1   6.977728
6      0.068774 -0.028343       7        1   5.206019, topic_info=               Term         Freq        Total Category  logprob  loglift
38             plus  6667.000000  6667.000000  Default  30.0000  30.0000
1             titre  2442.000000  2442.000000  Default  29.0000  29.0000
5             texte  3078.000000  3078.000000  Default  28.0000  28.0000
63          pouvoir  4050.000000  4050.000000  Default  27.0000  27.0000
60            faire  4140.000000  4140.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
575      impression   203.512900   204.356029   Topic7  -5.0851   2.9512
560          mourir   200.469109   201.312524   Topic7  -5.1002   2.9512
995   international   200.554962   201.399168   Topic7  -5.0998   2.9512
803          moitie   195.679884   196.523482   Topic7  -5.1244   2.9511
1755          juste   193.447084   194.290380   Topic7  -5.1358   2.9510

[262 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
318       3  0.998434      agir
787       3  0.998774  ailleurs
1542      6  0.996171   ajouter
733       7  0.998089     alors
708       7  0.998244        an
...     ...       ...       ...
6         5  0.999374       vol
566       5  0.999573    volume
1924      3  0.998435       von
243       1  0.915375     œuvre
243       2  0.084013     œuvre

[256 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 6, 2, 3, 1, 7])

La visualisation pour le modèles 4 avec 7 thématiques vient confirmer notre hypothèse. Les thématiques deviennent moins parlantes et plus difficilement interprétables.  

### Conclusion : 

Nous pouvons alors retenir que avec un grand corpus de texte traitant de vastes sujets mais préservant aussi un vocabulaire technique qui les lient entre eux un grand nombre de thématique semble être le plus adaptées pour identifiées les variations entres les textes. La LDA permet ainsi d'embrasser les différents sujet sur lesquels portent les comptes rendus tout en dégageant leur similarités. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e45e14b3-d43d-4495-8c6b-f8348ef65aef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>